# Setup

Run the follwing cell to pip install the necerssary packages specified in the requirements.txt file.

In [ ]:
pip install -r requirements.txt

# to push/pull form ucloud enter following code in terminal
git config --global user.name "FIRST_NAME LAST_NAME"
git config --global user.email "MY_NAME@example.com"


Importing the necessary packages

In [ ]:
import os
import io
import tensorflow as tf
import numpy as np
from azure.storage.blob import BlobServiceClient, ContainerClient
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns
import ast
import time
from keras.models import load_model

import pandas as pd
from tensorflow.keras import layers, models
import tensorflow_hub as hub
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model, Sequential
from keras.utils import plot_model
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, confusion_matrix
from keras.callbacks import Callback, ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

### Connect to Azure

In [ ]:
#set up storage
connection_string = "DefaultEndpointsProtocol=https;AccountName=mlfinalexam5505462853;AccountKey=0c40lghglG5/GlNK9yujDQAgo38GKoS2I3DeC/g22hwAEIFANKpmC/TqOpRk4RCT1DbfNiHBFt72+AStB+PfUA==;EndpointSuffix=core.windows.net"
container_name = "meterml"

#create client
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
container_client = blob_service_client.get_container_client(container_name)

### Load Image Paths and Labels

In [ ]:
#get filepaths
df = pd.read_csv("FINAL_METER_ML_test.csv")


# convert each string in the DataFrame to a list
df['Label'] = df['Label'].apply(ast.literal_eval)

# convert each list in the DataFrame to a numpy array
df['Label'] = df['Label'].apply(np.array)

class_names=["CAFOs","Landfills","Mines","Negative","ProcessingPlants","RefineriesAndTerminals","WWTreatment"]

# Helper Functions 

- Training Accuracy and Loss Graphs: plot_history(history)
- Predictions: print_predictions(inceptionv3_model, test_ds)
- True and Predicted Classes: true_classes,predicted_classes = true_pred_classes(inceptionv3_model, test_ds)
- Accuracy: accuracy_score(true_classes,predicted_classes)
- F1 Score: f1_score(true_classes, predicted_classes, average='weighted')
- Recall: recall = recall_score(true_classes, predicted_classes, average='weighted')
- Precision: precision = precision_score(true_classes, predicted_classes, average='weighted')
- Confusion MAtrix (as Array): conf_matrix = confusion_matrix(true_classes, predicted_classes)
- Plot Confusion Matrix: print_conf_matrix(true_classes, predicted_classes,class_names)

In [ ]:
image_size=224
channels=3
autotune = tf.data.experimental.AUTOTUNE # Adapt preprocessing and prefetching dynamically

def data_split(df):
    """Splits and returns the dataset into training, validation, and test"""
    X_temp, X_test, y_temp, y_test = train_test_split(df['Image_Folder'], df['Label'], test_size=0.15, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.176, random_state=42)
    #convert labels to array
    y_train = np.array(y_train).tolist()
    y_val = np.array(y_val).tolist()
    y_test = np.array(y_test).tolist()
    #print number of observations per datasets
    print("Nr. Training:",len(X_train),"Nr. Validation:",len(X_val),"Nr. Test:",len(X_test))
    
    return X_train, X_val, X_test, y_train, y_val, y_test


def load_image(path):
    """Load an image from Azure Blob Storage."""
    blob_client = container_client.get_blob_client(path)
    blob_data = blob_client.download_blob().readall()  # Directly read all bytes
    
    return io.BytesIO(blob_data)


def load_and_preprocess_image(path):
    """Loads an image, decodes it to grayscale, resizes, and normalizes it."""
    # Load image
    image_file = load_image(path.numpy().decode('utf-8'))
    # Decode the image to grayscale
    image_tensor = tf.io.decode_image(image_file.getvalue(), channels=channels)
    # Resize the image
    image_resized = tf.image.resize(image_tensor, [image_size, image_size])
    # Normalize the image data
    image_normalized = image_resized / 255.0
    return image_normalized


def process_tensor(path, label):
    """Function to load an image from blob storage, decode, resize, and normalize it."""
    image_normalized = tf.py_function(load_and_preprocess_image, [path], tf.float32)
    # Ensure the shape is set correctly for grayscale
    image_normalized.set_shape([image_size, image_size, channels])
    return image_normalized, label


def create_dataset(filenames, labels, is_training=True):
    """Creates a TensorFlow dataset from filenames and labels."""
    dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))
    dataset = dataset.map(process_tensor, num_parallel_calls=tf.data.AUTOTUNE)
    #shuffle the data when it is the training dataset
    if is_training:
        dataset = dataset.cache()
        dataset = dataset.shuffle(buffer_size=1024)
    #creates batches    
    dataset = dataset.batch(256)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)

    return dataset


def create_all_datasets(X_train, X_val, X_test, y_train, y_val, y_test ):
    """Creates train, test, and val datasets by calling the create_dataset function each."""
    train_ds = create_dataset(X_train, y_train)
    test_ds = create_dataset(X_test, y_test, False)
    val_ds = create_dataset(X_val, y_val, False)
    
    return train_ds, test_ds, val_ds


def print_dataset(dataset):
    """Print the plain dataset."""
    for images, labels in dataset.take(1):  # Here, take(1) takes the first batch
        print("Images:", images.numpy())  # Convert tensor to numpy array and print
        print("Labels:", labels.numpy())  # Convert tensor to numpy array and print


def plot_history(model):
    """Plots the accuracy and loss of the inputted model."""
    # summarize history for accuracy
    plt.plot(model.history['accuracy'])
    plt.plot(model.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()
    
    # summarize history for loss
    plt.plot(model.history['loss'])
    plt.plot(model.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()


def print_predictions(model, ds):
    """Predictions based on test dataset."""
    #predict
    for images, labels in ds:
        predictions = model.predict(images)  # Only pass image data
        classes = predictions.argmax(axis=-1) #selects biggest value as prediction

        for pred, classe, label in zip(predictions,classes, labels):
            print("Prediction:", pred,"Pred. Class: ",classe, "Actual Label:", label.numpy())# Print the first prediction
        break
    
        
def plot_model(model): 
    """Plot model with predefined arguments."""
    plot_model(model, 
            to_file='vgg.png',
            show_shapes=True,
            show_dtype=True,
            show_layer_names=True,
            show_layer_activations=True,
            show_trainable=False)
    

def evaluate_model(model, test_ds):
    result = model.evaluate(test_ds)
    # Assuming accuracy was the second metric (index 1), extract the accuracy.
    test_accuracy = result[1] * 100  # Convert to percentage
    print(f"Test Accuracy: {test_accuracy:.2f}%")
    return test_accuracy


def true_pred_classes(model, dataset): 
    """
    Evaluates the given model using the dataset.
    Returns: accuracy, f1, recall, precision, confusion matrix
    """
    # Collect all labels and predictions
    true_classes = []
    predicted_classes = []

    # Iterate over the dataset
    for images, labels in dataset:
        # Predict batch
        preds = model.predict(images)
        preds = np.argmax(preds, axis=1)
        true = labels.numpy()  # Assuming labels are already integer-encoded

        # Append to lists
        true_classes.extend(true)
        predicted_classes.extend(preds)
    return true_classes,predicted_classes


def print_conf_matrix(true_classes, predicted_classes, class_names):
    """
    Print confusion matrix.
    """
    conf_matrix = confusion_matrix(true_classes, predicted_classes)
    df_cm = pd.DataFrame(
        conf_matrix, index=class_names, columns=class_names,
    )
    fig = plt.figure(figsize=(10,7))
    heatmap = sns.heatmap(df_cm, annot=True, fmt="d", cmap='Blues')

    # Set aesthetics for better readability
    heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right', fontsize=14)
    heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right', fontsize=14)

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.title('Confusion Matrix')
    plt.show()


# Create Datasets

In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test = data_split(df)
    
train_ds, test_ds, val_ds = create_all_datasets(X_train, X_val, X_test, y_train, y_val, y_test)

# Models

## ResNet50: 
- https://datagen.tech/guides/computer-vision/resnet-50/
- https://medium.com/@bravinwasike18/building-a-deep-learning-model-with-keras-and-resnet-50-9dd6f4eb3351
- https://medium.com/@ozgunhaznedar/image-classification-on-satellite-images-with-deep-learning-baa9813dde4e
- https://wandb.ai/mostafaibrahim17/ml-articles/reports/The-Basics-of-ResNet50---Vmlldzo2NDkwNDE2#step-4:-building-resnet-50-model

### Transfer Learning

In [ ]:
#set variables
train_epochs = 30
tune_epochs = 10
total_epochs = train_epochs + tune_epochs
batch_size = 128

# import resnet model for transfer learning
rn50_base = tf.keras.applications.ResNet50(
    include_top = False,
    weights = "imagenet",
    input_shape=(224,224,3)
    )

# Freeze layers pf basemodel, so the pre-trained weights are fixed
for each_layer in rn50_base.layers:
        each_layer.trainable=False

# create sequential model
resnet_model = Sequential()

# Add output layers for finetuning
resnet_model.add(rn50_base)
resnet_model.add(Flatten()) #use flatten instead of GlobalAveragePooling2D as it may yield better results when enough data
resnet_model.add(Dense(512, activation='relu'))
resnet_model.add(Dense(7, activation='softmax'))

# Compile model
resnet_model.compile(optimizer= tf.keras.optimizers.Adam(),
                    loss= tf.keras.losses.CategoricalCrossentropy(), 
                    metrics = ['accuracy'])

#initializt time
time.time()

# Train model 
history = resnet_model.fit(train_ds, 
                        validation_data = val_ds, 
                        epochs = train_epochs, 
                        batch_size=batch_size, 
                        callbacks = [ReduceLROnPlateau(patience=5), EarlyStopping(patience=10)])

#print time in seconds
print("Training time in seconds:", time.time()-t0)

#save model
resnet_model.save("resnet_model.keras")

# Plot model information
plot_history(history)

### Fine Tuning

In [ ]:
#unfreeze last convolution layer for fine tuning
for each_layer in rn50_base.layers:
        each_layer.trainable=False
for layer in [l for l in rn50_base.layers if 'conv5' in l.name]:
   layer.trainable = True
   
for i, layer in enumerate(rn50_base.layers):
    print(i, layer.name, "-", layer.trainable)

In [ ]:
#compile the model with smaller learning rate
resnet_model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0001),
                    loss = tf.keras.losses.CategoricalCrossentropy(), 
                    metrics=['accuracy'])

#initialize timing
t0 = time.time()

# Train model 
history = resnet_model.fit(train_ds, 
                        validation_data = val_ds, 
                        epochs=total_epochs, 
                        batch_size=batch_size, 
                        callbacks = [ReduceLROnPlateau(patience=5), EarlyStopping(patience=10)])

print("Training time in seconds:", time.time()-t0)

#save trained model
resnet_model.save("resnet_tf_model.keras")

# Plot training plots
plot_history(history)

## Self-trained VGG16
Source: https://medium.com/@siddheshb008/vgg-net-architecture-explained-71179310050f

In [ ]:
#Set Variables
epochs = 50
batch_size = 128

# Define Model
_input = Input((224,224,3)) 

conv1  = Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu")(_input)
conv2  = Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu")(conv1)
pool1  = MaxPooling2D((2, 2))(conv2)

conv3  = Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu")(pool1)
conv4  = Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu")(conv3)
pool2  = MaxPooling2D((2, 2))(conv4)

conv5  = Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu")(pool2)
conv6  = Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu")(conv5)
conv7  = Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu")(conv6)
pool3  = MaxPooling2D((2, 2))(conv7)

conv8  = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(pool3)
conv9  = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(conv8)
conv10 = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(conv9)
pool4  = MaxPooling2D((2, 2))(conv10)

conv11 = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(pool4)
conv12 = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(conv11)
conv13 = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(conv12)
pool5  = MaxPooling2D((2, 2))(conv13)

flat   = Flatten()(pool5)
dense1 = Dense(4096, activation="relu")(flat)
dense2 = Dense(4096, activation="relu")(dense1)
output = Dense(7, activation="softmax")(dense2) #adapted number of outputs and outputfunction

vgg16_model  = Model(inputs=_input, outputs=output)


#compile the model
vgg16_model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

##provide a model summary
vgg16_model.summary()

#initialize timing
time.time()

#fit the model
vgg16_model.fit(
    train_ds,
    validation_data = val_ds,
    epochs=epochs,
    batch_size=batch_size,
    callbacks = [ReduceLROnPlateau(patience=5), EarlyStopping(patience=10)])

print("Training time in seconds:", time.time()-t0)

vgg16_model.save("vgg16_model.keras")

plot_history(vgg16_model)


## Alexnet New

In [ ]:
IMG_SIZE = 224
CHANNELS = 3

alexnet = Sequential()

alexnet.add(Conv2D(96, kernel_size=(11,11), strides= 4,
                        padding= 'valid', activation= 'relu',
                        input_shape= (224,224,3),
                        kernel_initializer= 'he_normal'))

alexnet.add(MaxPooling2D(pool_size=(3,3), strides= (2,2),
                              padding= 'valid', data_format= None))

alexnet.add(Conv2D(256, kernel_size=(5,5), strides= 1,
                        padding= 'same', activation= 'relu',
                        kernel_initializer= 'he_normal'))
                        
alexnet.add(MaxPooling2D(pool_size=(3,3), strides= (2,2),
                              padding= 'valid', data_format= None)) 

alexnet.add(Conv2D(384, kernel_size=(3,3), strides= 1,
                        padding= 'same', activation= 'relu',
                        kernel_initializer= 'he_normal'))

alexnet.add(Conv2D(384, kernel_size=(3,3), strides= 1,
                        padding= 'same', activation= 'relu',
                        kernel_initializer= 'he_normal'))

alexnet.add(Conv2D(256, kernel_size=(3,3), strides= 1,
                        padding= 'same', activation= 'relu',
                        kernel_initializer= 'he_normal'))

alexnet.add(MaxPooling2D(pool_size=(3,3), strides= (2,2),
                        padding= 'valid', data_format= None))

alexnet.add(Flatten())
alexnet.add(Dense(4096, activation= 'relu'))
alexnet.add(Dense(4096, activation= 'relu'))
alexnet.add(Dense(1000, activation= 'relu'))

alexnet.add(Dense(7, activation= 'softmax'))

# Compile the model:
alexnet.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                loss=tf.keras.losses.CategoricalCrossentropy(),
                metrics=['accuracy'])

#initialize timing
time.time()

# Fit model and save history for further analysis:
history = alexnet.fit(train_ds,
                      validation_data = val_ds, 
                      epochs=30,
                      batch_size=128,
                      callbacks=[ReduceLROnPlateau(patience=5), tf.keras.callbacks.EarlyStopping(patience=5)])

print("Training time in seconds:", time.time()-t0)

alexnet.save('alexnet.keras')

plot_history(history)